# Pytorch中的神经网络基本单元

In [1]:
import torch
from torch import nn

# nn.Module

nn.Module是神经网络结构的表示，它可以表示一个层，也可以表示一个结构块，也可以表示一个完整的模型结构。

> Modules can also contain other Modules, allowing to nest them in a tree structure. 

## 自定义一个layer

In [2]:
class ReluLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.maximum(x, torch.tensor(0))

relu = ReluLayer()
print(relu)

ReluLayer()


In [3]:
a = torch.randn(2, 3)
relu(a)

tensor([[0.0000, 1.4337, 2.0141],
        [0.6990, 0.0000, 1.0046]])

如果我们自定义的 "Layer" 中有需要训练的参数，需要定义为 `nn.Parameter`类型

In [4]:
# 带参数的Layer
class MyFCLayer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(in_dim, out_dim))
        self.bias = nn.Parameter(torch.randn(out_dim))

    def forward(self, x):
        return x.matmul(self.weights.data) + self.bias.data


fclayer = MyFCLayer(25, 10)
print(fclayer)

MyFCLayer()


In [5]:
a = torch.randn(1, 25)
fclayer(a)

tensor([[  4.6710,   6.3528,   3.4600,   7.2031,  -0.0789,  -4.6998,  -1.9341,
         -16.8910,  -7.7743,  -4.8531]])

## 自定义一个Block

一般情况下一个 Block 由有若干个 Layer 形成的一种特别的结构，比如 TransformerBlock, VGGBlock 等。

In [6]:
class LinearReluStack(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 100),
            ReluLayer(),
            nn.Linear(100, 100),
            ReluLayer(),
            MyFCLayer(100, 10),
        )

    def forward(self, x):
        return self.stack(x)


linear_relu_stack = LinearReluStack()
print(linear_relu_stack)

LinearReluStack(
  (stack): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReluLayer()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReluLayer()
    (4): MyFCLayer()
  )
)


In [7]:
a = torch.randn(1, 28 * 28)
linear_relu_stack(a)

tensor([[ 1.9470, -2.1320, -2.2663, -2.2337,  2.5418, -3.2080, -3.0094, -2.6686,
         -1.1289, -0.5292]], grad_fn=<AddBackward0>)

## 自定义一个模型

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.stack = LinearReluStack()

    def forward(self, x):
        return self.stack(self.flatten(x))


model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (stack): LinearReluStack(
    (stack): Sequential(
      (0): Linear(in_features=784, out_features=100, bias=True)
      (1): ReluLayer()
      (2): Linear(in_features=100, out_features=100, bias=True)
      (3): ReluLayer()
      (4): MyFCLayer()
    )
  )
)


In [9]:
a = torch.randn(1, 28, 28)
model(a)

tensor([[-2.4962,  1.1806, -0.8129, -0.5095, -0.9446, -2.6407, -0.1641,  0.6045,
         -1.3634, -1.4108]], grad_fn=<AddBackward0>)

# 参数

每一层的参数，我们可以通过`layer.bias`和`layer.weight`来访问，得到的是一个`nn.parameter.Parameter`的类型对象。

对于Sequential的模型，我们可以通过下标来访问每一层：`seqmodel[i]`

我们也可以通过`state_dict`来获取nn.Module中的所有层的参数。

In [10]:
mlp = nn.Sequential(nn.Linear(25, 100), nn.ReLU(), nn.Linear(100, 10))
# 通过索引获取 nn.Sequential 中的 Module
first_layer = mlp[0]
# 可以直接访问对应 module 的属性
first_layer.bias
first_layer.weight

# state_dict 返回该 module 以及所有子 module 的状态数据（parameter & buffer)
first_layer.state_dict()
type(mlp.state_dict())

collections.OrderedDict

In [11]:
# 获取所有参数
print(*[(name, param.shape) for name, param in mlp.named_parameters()])

('0.weight', torch.Size([100, 25])) ('0.bias', torch.Size([100])) ('2.weight', torch.Size([10, 100])) ('2.bias', torch.Size([10]))


In [12]:
# 访问OrderedDict
mlp.state_dict()["2.weight"].shape

torch.Size([10, 100])

对于`nn.parameter.Parameter`类型的对象，我们可以通过`.data`与`.grad`拿到其数据与梯度。

In [13]:
first_layer.bias.shape, first_layer.bias.grad  # (torch.Size([100]), None)

(torch.Size([100]), None)

# 参数初始化

对整个网络应用某个初始化函数

In [14]:
def norm_init(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)


mlp = mlp.apply(norm_init)

单独的某层layer应用初始化 

In [15]:
def xiaver_init(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)


mlp[2].apply(xiaver_init)

Linear(in_features=100, out_features=10, bias=True)

# 多个layer共享参数

In [16]:
shared = nn.Linear(8, 8)  # 需要共享参数的layer
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1)
)

net[2]和net[4]是共享参数的，梯度累加。 共享的 module 在整个模型中获取 modules 或者 parameters 列表时，不会重复列出，内部会去重。

In [17]:
for name, mod in net.named_children():
    print (mod)

Linear(in_features=4, out_features=8, bias=True)
ReLU()
Linear(in_features=8, out_features=8, bias=True)
ReLU()
ReLU()
Linear(in_features=8, out_features=1, bias=True)


# nn.Module 常见接口

## buffers 和 parameters

<img src="../images/torch_module_api.png" width=550px>

buffers 和 parameters 是Module 中常见的两类数据成员，parameters 保存的是模型训练的参数，它会在反向传播中进行更新，另一类不需要计算梯度的数据是 buffers，一般主要用于前向计算中，比如 BatchNorm 中的 moving_mean 和 moving_var，以及在位置编码中的 cos 和 sin 频率表。

buffers 和 parameters 本质上都是 `torch.Tensor`，只所以这里包装成了 buffers 和 parameters 两个概念，主要是方便`Module`对于这两种类型数据成员的管理，比如：可以通过接口一次性获取到所有子 Modules 的该类型的数据成员。

我们可以通过 `nn.Modules` 的 `buffers`和`parameters`接口获取模块以及其子模块中保存的所有数据成员。或者通过 `named_buffers`以及`named_parameters`来同时获取对应数据成员的名称。

In [18]:
batch_norm = nn.BatchNorm2d(num_features=128)

print("Buffers:")
for name, buffer in batch_norm.named_buffers():
    print(f"\t{name}: {buffer.shape}")
print("Parameters:")
for name, param in batch_norm.named_parameters():
    print(f"\t{name}: {param.shape}")

Buffers:
	running_mean: torch.Size([128])
	running_var: torch.Size([128])
	num_batches_tracked: torch.Size([])
Parameters:
	weight: torch.Size([128])
	bias: torch.Size([128])


我们在实现像 `BatchNorm` 这样的类时，我们可以调用 `register_buffer` 来创建 buffer 类型的数据成员。

```python
class BatchNorm2D(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.mean = nn.Parameter(torch.randn(num_features))
        self.var = nn.Parameter(torch.randn(num_features))
        self.register_buffer("moving_mean", torch.randn(num_features))
        self.register_buffer("moving_var", torch.randn(num_features))
```

经过 `register_buffer` 注册过的 buffer 成员，会自动的被 `named_buffers`这样的接口以及`state_dict`这样的接口捕获到。从上面的代码中，也可以看出对于`nn.Parameter`类型的数据，在构造函数中会被自动捕获到，如果某些参数是动态添加到 Module 当中的，则需要使用`register_parameter`。

以下是 `nn.Parameter` 类型的成员自动注册到 `Module` 的`_parameters`详细流程：

1. **创建 `nn.Parameter`**： 当你在 `nn.Module` 中定义一个参数时，如 `self.weight = nn.Parameter(torch.randn(10, 10))`，会创建一个 `Parameter` 对象。
2. **调用 `Module.__setattr__`**： 当将这个 `Parameter` 赋值给模型（即 `self.weight = nn.Parameter(...)`），会触发 `Module` 的 `__setattr__` 方法。
3. **注册到 `_parameters`**： 在 `__setattr__` 中，PyTorch 检查这个属性是否为 `Parameter` 类型。如果是，它会将这个参数的名称（例如 `weight`）和对应的 `Parameter` 对象一起注册到模型的 `_parameters` 字典中。

## `apply` 调用

Module 的 `apply(fn)` 函数用于在 Module 的所有子 Module应用一个自定义的函数。该函数常常用于我们来对模型的参数对应一些自定义的初始化。

在`nn.Module`的源码里，还有一个`_apply`函数，它与`apply`的区别是，它的自定义函数是针对于 Module 中的数据成员的，也就是 parameters 和 buffers，比如当我们调用`model.cuda()`时，其内部实际上调用的就是`_apply`函数，将 Module 中的所有数据成员转换到 GPU 上。

```python
self._apply(lambda t: t.cuda(device))
```

In [19]:
def norm_init(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)


model = nn.Linear(100, 100)
model = model.apply(norm_init)

model = model.cuda()

# Containers

`nn.ModuleList` 和 `nn.ModuleDict` 本身没有实现 `forward` 函数。这是因为它们只是用于存储一组子模块的容器，而不是一个完整的可执行模块。它们的主要作用是帮助你将多个子模块组织在一起，并自动注册到父模块中，但如何对这些子模块进行前向传播需要你在父类的 `forward` 函数中手动定义。

## nn.Sequential

`nn.Sequential` 是 PyTorch 中最常见的容器之一，通常用于将多个层（例如线性层、激活函数、卷积层等）按顺序连接。它的作用类似于一个列表，逐步将输入通过各个层传递。

当网络结构相对简单时，可以利用 `nn.Sequential` 快速地构建网络。 在某些简单任务中，可以避免手动定义 forward 函数。

In [20]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Linear(32 * 14 * 14, 10),  # 假设输入图像大小为28x28
)

模仿nn.Sequential

In [21]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, x):
        # _modules是内部的一个OrderedDict
        for module in self._modules.values():
            x = module(x)
        return x


mlp = MySequential(nn.Linear(25, 100), nn.ReLU(), nn.Linear(100, 10))
print(mlp)

MySequential(
  (0): Linear(in_features=25, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)


## nn.ModuleList

`nn.ModuleList` 用于存储一组子模块。与 Python 中的列表类似，但 ModuleList 中存储的模块会被 PyTorch 正确注册为子模块。这意味着它们的参数会被自动添加到父模块中，并能通过 `model.parameters()` 访问。它适用于那些模块数量或结构动态变化的情况。

比如我们希望实验一个 50 层的全连接网络的梯度消失与梯度爆炸的问题，可以用 `ModuleList`来快速定义一个 50 层的模型。

In [22]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(100, 100) for _ in range(50)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

## nn.ModuleDict

`nn.ModuleDict` 与 Python 的字典类似，它将子模块以键值对的形式存储。与 `ModuleList` 不同的是，`ModuleDict` 的模块可以通过键名访问，适合需要灵活地通过名称调用特定模块的场景。

In [23]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleDict(
            {"conv": nn.Conv2d(1, 32, 3, 1), "fc": nn.Linear(32 * 26 * 26, 10)}
        )

    def forward(self, x):
        x = self.layers["conv"](x)
        x = x.view(x.size(0), -1)
        x = self.layers["fc"](x)
        return x